Import and constants

In [1]:
from torch import nn, optim, float32, IntTensor, FloatTensor
from dataset.chestxray import ChestXRayDataset
from datahandling.transforms import to_numeric_label
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models
from torchvision.io import read_image, ImageReadMode
import torchvision.transforms as transforms
import os
import pandas as pd
import torchvision
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
debug = False

Important paths

In [4]:
# Names and base paths, define here and combine
db_path = "/cluster/home/larsira/tdt4900/databases/chest_xray14/"
img_dir_name = "images"
annotations_file = os.path.join(db_path, "data_list.csv")

img_dir = os.path.join(db_path, img_dir_name)

Sanity check data

In [5]:
# Load the first image and see what is up
image = Image.open(os.path.join(img_dir, "00000001_000.png"))

# Load annotations file and inspect a label
annotations = pd.read_csv(os.path.join(db_path, annotations_file))
print(os.path.join(db_path, annotations_file))

if debug:
    print(annotations.iloc[1])
    print("This patient is aflicted with:", annotations["Finding Labels"])
    print(image)
    plt.imshow(image)

FileNotFoundError: [Errno 2] No such file or directory: '/cluster/home/larsira/tdt4900/databases/chest_xray14/images\\00000001_000.png'

Fetch relevant labels from dataset

In [5]:
labels = set([x for y in [x.split("|") for x in annotations["Finding Labels"]] for x in y])
labels.remove("No Finding")
print(labels, "There are", len(labels), "labels available")

{'Pneumonia', 'Fibrosis', 'Atelectasis', 'Emphysema', 'Effusion', 'Pleural_Thickening', 'Infiltration', 'Pneumothorax', 'Cardiomegaly', 'Nodule', 'Hernia', 'Mass', 'Consolidation', 'Edema'} There are 14 labels available


Load into dataset, define a data loader

In [8]:
batch_size = 8
transform = transforms.Compose(
    [transforms.ConvertImageDtype(float32),
    transforms.Normalize((0.5,), (0.5,))])
dataset = ChestXRayDataset(annotations_file, img_dir, transform, to_numeric_label)
loader = DataLoader(dataset, batch_size = batch_size)

if debug:
    for idx, value in enumerate(loader):
        test_img, test_lab = value
        img = test_img[0]
        label = test_lab[0]
        if idx == 100:
            break

torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])


libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024,

So here, we should probably define ourselves a model

In [9]:
model = models.resnet101(weights=None)
model.fc = nn.Linear(2048, len(labels))

In [10]:
# Hyper params
lr = 10e-4

optimizer = optim.SGD(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

def train_epoch(idx, data_loader):
    prev_loss = 0
    running_loss = 0
    
    for i, data in enumerate(data_loader):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000
            print("batch {} loss: {}".format(i+1, last_loss))
            running_loss = 0.
    
    return last_loss

In [ ]:
epochs = 10
best_loss = 10_000_000.

for epoch in range(epochs):
    model.train(True)

    avg_loss = train_epoch(epoch, loader)

    running_validation_loss = 0
    model.eval()
    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            validation_inputs, validation_labesl = vdata
            validation_output = model(validation_inputs)
            validation_loss = loss_fn(validation_outputs, validation_inputs)
            running_validation_loss += validation_loss
    avg_validation_loss = running_validation_loss/(i+1)

    if avg_validation_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), "model_{}_{}".format(epoch, epoch))

torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])


libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024, 1024])
torch.Size([3, 1024,